# 1.0 Fix data

AIMS

En este archivo se realiza la limpieza de los archivos y el ETL

In [19]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
import pandas as pd
from datetime import datetime
from mypackage import dir, dataquality
from mypackage.transforms import non_sex, diff_date #convertir_variables
from rfm import rfm

modality =  modality = 'p'
project = 'australia'
data = dir.make_dir_line(modality, project) 
raw = data('raw')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import warnings
warnings.filterwarnings("ignore")

In [21]:
def convertir_variables(dataframe, list_columns, type):
    '''
    Esta función reemplaza los tipos de variables
    '''
    for columna in list_columns:
        dataframe[columna] = dataframe[columna].astype(type)
    return dataframe

## 1.1 Fix data customers

In [45]:
df_clientes = pd.read_csv(raw / 'df_clientes.csv', sep = ',', decimal = '.', header = 0)
df_clientes['fecha_nacimiento'] = pd.to_datetime(df_clientes['fecha_nacimiento'], dayfirst='false', errors = 'coerce')
df_clientes = convertir_variables(df_clientes, ['id_cliente', 'sexo'], 'str')
df_clientes = non_sex(df_clientes, 'sexo')
df_clientes = diff_date(df_clientes, "fecha_nacimiento", 'now')
dataquality.data_quality(df_clientes, threshold_discrete=10)

Execution data quality starts.

*****
The dataframe does not contain missing values.
*****
The dataframe does not contain duplicate values.
*****
The dataframe contains values that are not numeric.
Variable not numerit: id_cliente
Variable not numerit: nombre
Variable not numerit: fecha_nacimiento
*****
Discrete variable: sexo
*****
The dimension of dataframe is (100, 5)
*****

Execution data quality ends.



In [46]:
df_clientes.head()

,id_cliente,nombre,sexo,fecha_nacimiento,age_days
0,1001,Fredo Bonanno-Beccheria,1,1930-05-14,34263
1,1002,Ermenegildo Sansoni,1,1940-09-19,30482
2,1003,Antonia Bajardi,0,1916-05-18,39372
3,1004,Evangelista Biagi,0,1985-01-20,14288
4,1005,Iolanda Ceschi,0,2016-06-24,2810


In [47]:
df_customers = df_clientes.copy()
df_customers = df_customers.loc[:,['id_cliente', 'sexo', 'age_days']]
df_customers = convertir_variables(df_customers, ['id_cliente', 'sexo'], 'str')
df_customers.head()

,id_cliente,sexo,age_days
0,1001,1,34263
1,1002,1,30482
2,1003,0,39372
3,1004,0,14288
4,1005,0,2810


## 1.2 Fix data products

In [59]:
df_productos = pd.read_csv(raw / 'df_productos.csv', sep = ',', decimal = '.', header = 0)
df_productos.rename(columns={'precio':'valor', 'production':'procentaje_produccion'}, inplace=True)
df_productos['valor_produccion'] = (df_productos['valor'] * df_productos['procentaje_produccion'])/100
dataquality.data_quality(df_productos, threshold_discrete=10)

Execution data quality starts.

*****
The dataframe does not contain missing values.
*****
The dataframe does not contain duplicate values.
*****
The dataframe contains values that are not numeric.
Variable not numerit: id_producto
Variable not numerit: articulo
Variable not numerit: category
Variable not numerit: color
Variable not numerit: L
Variable not numerit: tallas
*****
Discrete variable: tallas
*****
The dimension of dataframe is (361, 10)
*****

Execution data quality ends.



In [60]:
df_productos.head()

,id_producto,descripcion,articulo,category,valor,procentaje_produccion,color,L,tallas,valor_produccion
0,759ne,759,Short con tiritas,Short,40.0,56,Negro,ne,1 al 4,22.4
1,759bl,759,Short con tiritas,Short,40.0,56,Blanco,bl,1 al 4,22.4
2,759az,759,Short con tiritas,Short,40.0,56,Azul,az,1 al 4,22.4
3,759ro,759,Short con tiritas,Short,40.0,56,Rojo,ro,1 al 4,22.4
4,759ra,759,Short con tiritas,Short,40.0,56,Rosa,ra,1 al 4,22.4


In [69]:
df_products = df_productos.copy()
df_products = df_products.loc[:,['id_producto', 'valor', 'valor_produccion']]
df_products.head()

,id_producto,valor,valor_produccion
0,759ne,40.0,22.4
1,759bl,40.0,22.4
2,759az,40.0,22.4
3,759ro,40.0,22.4
4,759ra,40.0,22.4


## 1.3 Fix data transactions

In [70]:
df_ventas = pd.read_csv(raw / 'df_ventas.csv', sep = ',', decimal = '.', header = 0)
df_ventas = df_ventas.loc[:,['id_factura', 'fecha', 'id_cliente', 'id_producto', 'quantity', 'id_store', 
                             'discount', 'per_discount2']]
df_ventas['fecha'] = pd.to_datetime(df_ventas['fecha'], dayfirst='false', errors = 'coerce')
df_ventas = convertir_variables(df_ventas, ['id_factura', 'id_cliente', 'id_store', 'discount'], 
                                'str')
dataquality.data_quality(df_ventas, threshold_discrete=10)

Execution data quality starts.

*****
The dataframe does not contain missing values.
*****
The dataframe does not contain duplicate values.
*****
The dataframe contains values that are not numeric.
Variable not numerit: id_factura
Variable not numerit: fecha
Variable not numerit: id_cliente
Variable not numerit: id_producto
Variable not numerit: id_store
Variable not numerit: discount
*****
Discrete variable: quantity
*****
The dimension of dataframe is (5447, 8)
*****

Execution data quality ends.



In [71]:
df_ventas.head()

,id_factura,fecha,id_cliente,id_producto,quantity,id_store,discount,per_discount2
0,1001,2021-02-17,1088,2521ro,1,0,0,0.0
1,1002,2021-02-21,1089,9160ro,2,2,1,5.0
2,1003,2021-02-22,1050,9173ne,2,0,0,0.0
3,1004,2021-02-24,1029,762bl,1,1,0,0.0
4,1005,2021-02-24,1009,9167ro,1,1,0,0.0


## 1.4 Unir todas las tablas

In [72]:
df_customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id_cliente  100 non-null    object
 1   sexo        100 non-null    object
 2   age_days    100 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 2.5+ KB


In [73]:
print(df_ventas.shape, df_products.shape)
df = df_ventas.merge(df_products, on = ['id_producto'])
print(df.shape)

(5447, 8) (361, 3)
(5447, 10)


In [80]:
df['total_factura'] = df['valor'] * df['quantity']
df['total_produccion'] = df['valor_produccion'] * df['quantity']
df['total_descuento'] = (df['total_factura'] * df['per_discount2'])/100
df['total_factura_con_descuento'] = df['total_factura'] - df['total_descuento'] 
df['ganancia'] = df['total_factura_con_descuento'] - df['total_produccion'] 
df.head()

,id_factura,fecha,id_cliente,id_producto,quantity,id_store,discount,per_discount2,valor,valor_produccion,total_factura,total_produccion,total_descuento,total_factura_con_descuento,ganancia
0,1001,2021-02-17,1088,2521ro,1,0,0,0.0,52.8,13.2,52.8,13.2,0.00,52.80,39.60
1,1226,2021-04-07,1008,2521ro,2,0,0,0.0,52.8,13.2,105.6,26.4,0.00,105.60,79.20
2,2703,2021-12-25,1022,2521ro,1,1,0,0.0,52.8,13.2,52.8,13.2,0.00,52.80,39.60
3,2837,2022-03-01,1099,2521ro,2,0,1,15.0,52.8,13.2,105.6,26.4,15.84,89.76,63.36
4,2991,2022-04-03,1048,2521ro,1,1,1,5.0,52.8,13.2,52.8,13.2,2.64,50.16,36.96


In [29]:
print('ok_')

ok_
